# Quantiles and histograms
By the end of this lecture you will be able to:
- calculate a quantile on a `DataFrame`
- calculate a quantile on an expression
- calculate and visualise a histogram

In [ ]:
import polars as pl

In [ ]:
csv_file = "../data/titanic.csv"

In [ ]:
df = pl.read_csv(csv_file)
df.head(3)

## Quantiles

### Quantiles on a `DataFrame`
We calculate quantiles on a `DataFrame` using the `quantile` function. To get the 90th percentile we do:

In [ ]:
df.quantile(0.9)

### Quantiles in an expression

We can also calculate quantiles as an expression

In [ ]:
(
    df
    .select(
        pl.col('Age').quantile(0.9)
    )
)

### Multiple quantiles
We can calculate multiple quantiles in an expression using a list comprehension. As expressions are run in parallel this allows us to calculate multiple quantiles in parallel

In [ ]:
quantile_list = [0.1,0.5,0.9]
(
    df
    .select(
        [
            pl.col('Age').quantile(q).alias(f"Age_quantile_{q}") for q in quantile_list
        ]
    )
)

To calculate multiple quantiles on multiple columns we can use `name.suffix` to avoid column name collisions.

In this example we calculate multiple quantiles on all of the floating point columns

In [ ]:
quantile_list = [0.1,0.5,0.9]
(
    df
    .select(
        [
            pl.col(pl.Float64).quantile(q).name.suffix(f"_quantile_{q}") for q in quantile_list
        ]
    )
)

As quantiles can be calculated using a fast-track algorithm on sorted columns it may be worth sorting the columns you are calculting multiple quantiles on first

In [ ]:
quantile_list = [0.1,0.5,0.9]
(
    df
    .select(
        pl.col(pl.Float64).sort()
    )
    .select(
        [
            pl.col(pl.Float64).quantile(q).name.suffix(f"_quantile_{q}") for q in quantile_list
        ]
    )
)

### Interpolation strategy for quantiles
We can use different interpolation strategies for calculating quantiles:
- nearest 
- higher 
- lower 
- midpoint
- linear

In [ ]:
df.select(
    pl.col('Age').quantile(0.25,interpolation='nearest').alias('Age_nearest'),
    pl.col('Age').quantile(0.25,interpolation='linear').alias('Age_linear'),
)

## Histogram

### Histogram method
We can calculate a histogram on a `Series`. Here we get a histogram of the `Age` column

In [ ]:
(
    df["Age"]
    .hist()
)

The columns of the output are:
- `breakpoint` this is the right-hand value of each bin
- `category` which is a categorical column showing the range of each bin
- `count` showing the number of values in each bin

We can instead pass a `list` to specify the `bins`

In [ ]:
(
    df["Age"]
    .hist(
        bins=list(range(0,100,10)),
    )
)

If we just want to visualise the histogram, we can use the `plot.bar` method. In this case we specify the number of bins

In [ ]:
(
    df
    ["Age"]
    .hist(
        bins=list(range(0,100,10)),
    )
    .plot
    .bar(
        x="category",
        y="count"
    )
)

# Exercises

In the exercises you will develop your understanding of:
- calculating quantiles
- using different interpolation methods

## Exercise 1 - calculating quantiles
Calculate the 25th,50th and 75th percentiles for the `Age` column. Output the results as 3 columns (with appropriate names) in a one row `DataFrame`

In [ ]:
qs = [0.25,0.5,0.75]
df = pl.read_csv(csv_file)
df.head(2)

In [ ]:
(
    df
    <blank>
)

Calculate the same percentiles for all of the numeric columns.


Visualise the histogram of the `Fare` column with bins of width £25 up to £400

# Solutions

## Solution to Exercise 1
Calculate the 25th,50th and 75th percentiles for the `Age` column. Output the results as 3 columns (with appropriate names) in a one row `DataFrame`

In [ ]:
qs = [0.25,0.5,0.75]
(
    df
    .select(
        [pl.col('Age').quantile(q).alias(f"Age_{q}_quantile") for q in qs]
    )
)

Calculate the same percentiles for all of the numeric columns.


In [ ]:
qs = [0.25,0.5,0.75]
(
    df
    .select(
        [
            pl.col([pl.Float64,pl.Int64]).quantile(q).name.suffix(f"_{q}_quantile") for q in qs
        ]
    )
)

Visualise the histogram of the `Fare` column with bins of width £25 up to £400

In [ ]:
(
    df
    ["Fare"]
    .hist(
        bins=list(range(0,400,25))
    )
    .plot
    .bar(
        x="category",
        y="count"
    )
)